
# **Xây dựng Model phân loại cảm xúc trong văn bản**

In [2]:
%env MLFLOW_TRACKING_URI=http://mlflow.tsc.vn
%env MLFLOW_TRACKING_USERNAME=user
%env MLFLOW_TRACKING_PASSWORD=mH680qJ2pCpH

env: MLFLOW_TRACKING_URI=http://mlflow.tsc.vn
env: MLFLOW_TRACKING_USERNAME=user
env: MLFLOW_TRACKING_PASSWORD=mH680qJ2pCpH


In [5]:
pip install pandas scikit-learn joblib mlflow matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## **1. Import các thư viện cần thiết**

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib
import os
import mlflow # Import MLflow
import mlflow.sklearn # Import MLflow Sklearn flavor
import matplotlib.pyplot as plt # Import để lưu biểu đồ

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## **2. Tải dữ liệu**

In [ ]:
# Định nghĩa đường dẫn tới file CSV
file_path = 'data.csv'

# Kiểm tra xem file có tồn tại không
if not os.path.exists(file_path):
    print(f"Lỗi: File không tìm thấy tại đường dẫn: {file_path}")
    exit()
# Tải dữ liệu từ file CSV vào DataFrame
try:
    df = pd.read_csv(file_path)
    print("Đã tải dữ liệu thành công!")
    print("5 dòng đầu tiên của dữ liệu:")
    print(df.head())
    print("\nThông tin dữ liệu:")
    df.info()
except Exception as e:
    print(f"Lỗi khi tải dữ liệu: {e}")
    # Thoát nếu không thể tải dữ liệu thực tế
    exit()

Đã tải dữ liệu thành công!
5 dòng đầu tiên của dữ liệu:
                       comment label  rate Unnamed: 3
0               Áo bao đẹp ạ!!   POS     5        NaN
1                  Tuyệt vời !   POS     5        NaN
2   2day ao khong giong trong.   NEG     1        NaN
3  Mùi thơm,bôi lên da mềm da.   POS     5        NaN
4            Vải đẹp, dày dặn.   POS     5        NaN

Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18183 entries, 0 to 18182
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comment     18183 non-null  object
 1   label       18183 non-null  object
 2   rate        18183 non-null  int64 
 3   Unnamed: 3  23 non-null     object
dtypes: int64(1), object(3)
memory usage: 568.3+ KB


## **3. Tiền xử lý dữ liệu**


In [5]:
# Loại bỏ các hàng có giá trị NaN trong cột 'comment' hoặc 'label'
df.dropna(subset=['comment', 'label'], inplace=True)

# Chuyển đổi nhãn về định dạng thống nhất (ví dụ: chữ hoa)
df['label'] = df['label'].astype(str).str.upper()

print(f"\nSố lượng mẫu sau khi loại bỏ NaN: {len(df)}")
print("Phân bố các nhãn:")
print(df['label'].value_counts())


Số lượng mẫu sau khi loại bỏ NaN: 18183
Phân bố các nhãn:
POS    6816
NEG    6669
NEU    4698
Name: label, dtype: int64


## **4. Chia tập dữ liệu Training và Testing**

In [6]:

# Định nghĩa features (X) và target (y)
X = df['comment']
y = df['label']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
# test_size=0.2 có nghĩa là 20% dữ liệu sẽ được dùng để kiểm tra
# random_state để đảm bảo kết quả chia dữ liệu là như nhau mỗi lần chạy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Số lượng mẫu trong tập huấn luyện: {len(X_train)}")
print(f"Số lượng mẫu trong tập kiểm tra: {len(X_test)}")


Số lượng mẫu trong tập huấn luyện: 14546
Số lượng mẫu trong tập kiểm tra: 3637


## **5. Vector hóa văn bản (TF-IDF Vectorization)**

In [ ]:
# --- BẮT ĐẦU MLFLOW RUN ---
with mlflow.start_run(run_name="Text_Sentiments_Classifier_Run") as active_run:
    run_id = active_run.info.run_id
    print(f"MLflow Run ID: {run_id}")

# Khởi tạo TF-IDF Vectorizer
max_features = 5000 # max_features giới hạn số lượng từ/cụm từ độc nhất được xem xét
min_df = 5 # min_df bỏ qua các từ xuất hiện quá ít (ví dụ: chỉ 1 lần)
stop_words='english' #loại bỏ các từ dừng tiếng Anh

# Ghi log các tham số của vectorizer vào MLflow
mlflow.log_param("vectorizer_max_features", max_features)
mlflow.log_param("vectorizer_min_df", min_df)

vectorizer = TfidfVectorizer(max_features=max_features, min_df=min_df, stop_words=None) # Giữ stop_words=None nếu dữ liệu tiếng Việt

# Học từ vựng từ tập huấn luyện và chuyển đổi văn bản thành ma trận TF-IDF
X_train_vec = vectorizer.fit_transform(X_train)

# Chỉ chuyển đổi tập kiểm tra bằng vectorizer đã được huấn luyện
X_test_vec = vectorizer.transform(X_test)

print(f"Kích thước ma trận TF-IDF tập huấn luyện: {X_train_vec.shape}")
print(f"Kích thước ma trận TF-IDF tập kiểm tra: {X_test_vec.shape}")

Kích thước ma trận TF-IDF tập huấn luyện: (14546, 1557)
Kích thước ma trận TF-IDF tập kiểm tra: (3637, 1557)


## **6. Xây dựng và huấn luyện Model**

In [ ]:
# Khởi tạo mô hình Logistic Regression
# Logistic Regression là một lựa chọn tốt cho bài toán phân loại văn bản
max_iter = 1000
random_state = 42

# Ghi log các tham số của model vào MLflow
mlflow.log_param("model_max_iter", max_iter)
mlflow.log_param("random_state", random_state)
mlflow.log_param("model_type", "LogisticRegression")

model = LogisticRegression(max_iter=max_iter, random_state=random_state)

# Huấn luyện mô hình trên dữ liệu đã được vector hóa
print("\nBắt đầu huấn luyện mô hình...")
model.fit(X_train_vec, y_train)
print("Huấn luyện mô hình hoàn tất!")


Bắt đầu huấn luyện mô hình...
Huấn luyện mô hình hoàn tất!


## **7. Đánh giá Model**

In [ ]:
# Dự đoán nhãn trên tập kiểm tra
y_pred = model.predict(X_test_vec)

# In báo cáo phân loại
print("\nBáo cáo phân loại:")
report = classification_report(y_test, y_pred, output_dict=True)
print(classification_report(y_test, y_pred))

# Ghi log các metrics chi tiết vào MLflow
mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
# for label, metrics in report.items():
#     if isinstance(metrics, dict):
#         for metric_name, value in metrics.items():
#             mlflow.log_metric(f"{label}_{metric_name}", value)

# In độ chính xác tổng thể
accuracy = accuracy_score(y_test, y_pred)
print(f"Độ chính xác tổng thể (Accuracy): {accuracy:.4f}")


Báo cáo phân loại:
              precision    recall  f1-score   support

         NEG       0.68      0.79      0.73      1334
         NEU       0.48      0.35      0.41       940
         POS       0.79      0.80      0.80      1363

    accuracy                           0.68      3637
   macro avg       0.65      0.65      0.64      3637
weighted avg       0.67      0.68      0.67      3637

Độ chính xác tổng thể (Accuracy): 0.6835


## **8. Xuất Model và Vectorizer**

In [ ]:
# Tên của mô hình sẽ xuất hiện trong Model Registry
model_name = "EmotionTextClassifier"

# Lưu model (Logistic Regression) vào MLflow dưới dạng artifact
# và đăng ký vào Model Registry
mlflow.sklearn.log_model(
    sk_model=model,
    artifact_path="sentiment_model", # Đường dẫn trong artifact store cho run này
    registered_model_name=model_name # Tên mô hình trong Model Registry
)
print(f"\nModel '{model_name}' đã được lưu và đăng ký vào MLflow.")

# Lưu vectorizer (TfidfVectorizer) vào MLflow dưới dạng artifact
# MLflow không có 'flavor' riêng cho vectorizer, nên ta dùng joblib và log như artifact bình thường
joblib.dump(vectorizer, "vectorizer.pkl")
mlflow.log_artifact("vectorizer.pkl", artifact_path="sentiment_model")
os.remove("vectorizer.pkl") # Xóa file tạm sau khi đã log
print(f"Vectorizer đã được lưu vào MLflow artifacts.")

# Lưu báo cáo phân loại dưới dạng file text làm artifact
with open("classification_report.txt", "w") as f:
    f.write(classification_report(y_test, y_pred))
mlflow.log_artifact("classification_report.txt")
os.remove("classification_report.txt")


print(f"MLflow Run '{active_run.info.run_name}' (ID: {run_id}) đã hoàn tất!")
print(f"Bạn có thể xem chi tiết run tại: {mlflow.get_tracking_uri()}/#/experiments/{active_run.info.experiment_id}/runs/{run_id}")
print(f"Artifacts của run này được lưu tại: {active_run.info.artifact_uri}")

# --- KẾT THÚC MLFLOW RUN ---

In [ ]:

# Tạo thư mục 'model' nếu chưa tồn tại
output_dir = 'model'
os.makedirs(output_dir, exist_ok=True)

# Định nghĩa đường dẫn để lưu model và vectorizer
model_path = os.path.join(output_dir, 'model.pkl')
vectorizer_path = os.path.join(output_dir, 'vectorizer.pkl')

# Lưu model đã huấn luyện
joblib.dump(model, model_path)
print(f"\nModel đã được lưu tại: {model_path}")

# Lưu vectorizer đã huấn luyện
joblib.dump(vectorizer, vectorizer_path)
print(f"Vectorizer đã được lưu tại: {vectorizer_path}")

# %% [markdown]


Model đã được lưu tại: model/model.pkl
Vectorizer đã được lưu tại: model/vectorizer.pkl


## **9. Ví dụ sử dụng Model đã lưu**

In [ ]:

print("\n--- Ví dụ sử dụng Model đã lưu từ MLflow ---")

# Tải lại model và vectorizer từ MLflow (sử dụng tên từ Model Registry)
# Tải phiên bản mới nhất ở giai đoạn Production (hoặc Staging, None)
# Bạn có thể chỉ định version cụ thể: "models:/EmotionTextClassifier/1"
try:
    loaded_model_from_mlflow = mlflow.pyfunc.load_model(f"models/{model_name}/Production")
    print(f"Đã tải model '{model_name}' từ MLflow Model Registry (Production).")
except Exception as e:
    print(f"Không thể tải model từ giai đoạn Production: {e}")
    print(f"Thử tải phiên bản mới nhất không theo giai đoạn: models:/{model_name}/latest")
    loaded_model_from_mlflow = mlflow.pyfunc.load_model(f"models:/{model_name}/latest")
    print(f"Đã tải model '{model_name}' từ MLflow Model Registry (latest version).")


# Để tải vectorizer, bạn cần truy cập artifact_uri của run đã log nó.
# Trong môi trường thực tế, bạn sẽ cần biết run_id của run đã log vectorizer đó.
# Để đơn giản cho ví dụ này, chúng ta sẽ tải lại nó từ file tạm nếu có,
# HOẶC bạn cần vào MLflow UI, tìm run tương ứng và sao chép artifact URI của vectorizer.pkl
# Giả sử chúng ta đang ở một run khác, để tải vectorizer đã lưu:
# Bạn cần tìm artifact_uri của run đã lưu vectorizer đó.
# Ví dụ: loaded_vectorizer_from_mlflow = joblib.load(mlflow.artifacts.download_artifacts(artifact_uri="runs:/<RUN_ID_CUA_RUN_TRUOC>/sentiment_model/vectorizer.pkl"))
# Vì trong ví dụ này model và vectorizer được log trong cùng 1 run,
# chúng ta có thể tái sử dụng 'run_id' từ 'active_run' nếu muốn tải lại ngay.
# Nhưng thực tế sau này, bạn sẽ tải chúng trong một script riêng, dựa vào run_id.

# Để tải vectorizer từ artifact của run hiện tại (cho ví dụ tức thì)
try:
    vectorizer_artifact_path = mlflow.artifacts.download_artifacts(
        run_id=run_id, # Sử dụng run_id của run hiện tại
        artifact_path="sentiment_model/vectorizer.pkl"
    )
    loaded_vectorizer_from_mlflow = joblib.load(vectorizer_artifact_path)
    print("Đã tải vectorizer từ MLflow artifacts.")
except Exception as e:
    print(f"Lỗi khi tải vectorizer từ MLflow artifacts: {e}")
    print("Vui lòng đảm bảo vectorizer.pkl tồn tại trong artifact store của run này.")
    exit() # Thoát nếu không thể tải vectorizer

# Thử dự đoán với một câu bình luận mới
new_comments = [
    "Sản phẩm rất tốt, tôi rất ưng ý!",
    "Chất lượng kém, không đáng tiền chút nào.",
    "Bình thường",
    "Chán, dùng như hạch",
    "Tuyệt vời, tôi rất ưng ý!",
    "nhàu nát, tôi sẽ trả lại",
    "tệ, khống đáng mua",
    "tạm ổn",
    "good, sẽ mua thêm cái nữa",
    "tuyệt vời, sẽ rủ người thân mua"
]

# Vector hóa các bình luận mới
new_comments_vec = loaded_vectorizer_from_mlflow.transform(new_comments)

# Dự đoán nhãn
predictions = loaded_model_from_mlflow.predict(new_comments_vec)

print("\nDự đoán cho các bình luận mới (tải từ MLflow):")
for comment, pred_label in zip(new_comments, predictions):
    print(f"- '{comment}' -> Nhãn dự đoán: {pred_label}")

In [23]:
# Tải lại model và vectorizer để kiểm tra
loaded_model = joblib.load(model_path)
loaded_vectorizer = joblib.load(vectorizer_path)

# Thử dự đoán với một câu bình luận mới
new_comments = [
    "Sản phẩm rất tốt, tôi rất ưng ý!",
    "Chất lượng kém, không đáng tiền chút nào.",
    "Bình thường",
    "Chán, dùng như hạch",
    "Tuyệt vời, tôi rất ưng ý!",
    "nhàu nát, tôi sẽ trả lại"
    "tệ, khống đáng mua",
    "tạm ổn",
    "good, sẽ mua thêm cái nữa",
    "tuyệt vời, sẽ rủ người thân mua"
]

# Vector hóa các bình luận mới
new_comments_vec = loaded_vectorizer.transform(new_comments)

# Dự đoán nhãn
predictions = loaded_model.predict(new_comments_vec)

print("\nDự đoán cho các bình luận mới:")
for comment, pred_label in zip(new_comments, predictions):
    print(f"- '{comment}' -> Nhãn dự đoán: {pred_label}")



Dự đoán cho các bình luận mới:
- 'Sản phẩm rất tốt, tôi rất ưng ý!' -> Nhãn dự đoán: POS
- 'Chất lượng kém, không đáng tiền chút nào.' -> Nhãn dự đoán: NEG
- 'Bình thường' -> Nhãn dự đoán: NEU
- 'Chán, dùng như hạch' -> Nhãn dự đoán: NEG
- 'Tuyệt vời, tôi rất ưng ý!' -> Nhãn dự đoán: POS
- 'nhàu nát, tôi sẽ trả lạitệ, khống đáng mua' -> Nhãn dự đoán: NEG
- 'tạm ổn' -> Nhãn dự đoán: NEU
- 'good, sẽ mua thêm cái nữa' -> Nhãn dự đoán: POS
- 'tuyệt vời, sẽ rủ người thân mua' -> Nhãn dự đoán: POS
